In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *

In [0]:
dbutils.widgets.text('incremental_flag','0')


In [0]:
incremental_flag = dbutils.widgets.get('incremental_flag')

In [0]:
%sql

select * from delta.`abfss://silver@azadlsgen7428.dfs.core.windows.net/car_sales`

In [0]:
df_source = spark.sql('select distinct(Date_ID) from delta.`abfss://silver@azadlsgen7428.dfs.core.windows.net/car_sales`')

In [0]:
if spark.catalog.tableExists('cars_catalog.gold.dim_date'):
    df_sink = spark.sql('select dim_date_key,Date_ID from cars_catalog.gold.dim_date')
else:
    df_sink = spark.sql('select 0 dim_date_key,Date_ID from delta.`abfss://silver@azadlsgen7428.dfs.core.windows.net/car_sales` where 1 = 0')

In [0]:
df_filter = df_source.join(df_sink,df_source.Date_ID == df_sink.Date_ID,'left').select(df_source.Date_ID,df_sink.dim_date_key)
df_filter.display()

In [0]:
df_filter_new = df_filter.filter(df_filter.dim_date_key.isNull()).select(df_filter.Date_ID)
df_filter_old = df_filter.filter(df_filter.dim_date_key.isNotNull())

In [0]:
if (incremental_flag == '0'):
    max_value = 1
else:
    if spark.catalog.tableExists('cars_catalog.gold.dim_date'):
        max_value_df = spark.sql('select max(dim_date_key) from cars_catalog.gold.dim_date')
        max_value = max_value_df.collect()[0][0]
if spark.catalog.tableExists('cars_catalog.silver.dim_date'):
    max_value_df = spark.sql('select max(dim_date_key) from cars_catalog.silver.dim_date')
    max_value = max_value_df.collect()[0][0]
else:
    max_value = 1

In [0]:

df_filter_new = df_filter_new.withColumn('dim_date_key',max_value+monotonically_increasing_id()+1)

In [0]:
df_final = df_filter_new.union(df_filter_old)


In [0]:
df_final_deduped = df_final.dropDuplicates(['dim_date_key'])
if spark.catalog.tableExists('cars_catalog.gold.dim_date'):
    delta_table = DeltaTable.forPath(spark,'abfss://gold@azadlsgen7428.dfs.core.windows.net/dim_date')
    delta_table.alias('t').merge(df_final_deduped.alias('s'),'t.dim_date_key = s.dim_date_key').whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
else:
    df_final_deduped.write.format('delta').mode('append').option('path','abfss://gold@azadlsgen7428.dfs.core.windows.net/dim_date').saveAsTable('cars_catalog.gold.dim_date')

In [0]:
%sql
select * from cars_catalog.gold.dim_date